In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model

import tensorflow as tf

%matplotlib inline

/home/training/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df_train = pd.read_csv("/data/MNIST/mnist_train.csv", header=None)
df_test = pd.read_csv("/data/MNIST/mnist_test.csv", header=None)

y_train = df_train.iloc[:, 0].values
y_test = df_test.iloc[:, 0].values
X_train = df_train.iloc[:, 1:].values/255
X_test = df_test.iloc[:, 1:].values/255


In [5]:
tf.__version__

'1.6.0'

In [39]:
tf.reset_default_graph()
tf.set_random_seed(1)

n_x = X_train.shape[1]
n_y = len(np.unique(y_train))
n_h = 100

learning_rate = 0.001

X = tf.placeholder(dtype=tf.float32, shape=(None, n_x))
y = tf.placeholder(dtype=tf.int32, shape=(None,))
print("X", X)
print("y", y)

y_oh = tf.one_hot(y, depth=n_y)
print("y_oh", y_oh)

def dense_layer(input, size, activation = tf.nn.relu):
    input_size = int(input.get_shape()[1])
    W = tf.Variable(tf.truncated_normal(shape=(input_size, size), stddev=0.1))
    b = tf.Variable(tf.zeros(shape=(size,)))
    Z = tf.matmul(input, W) + b
    
    if callable(activation):
        Z = activation(Z)
    return Z


A1 = dense_layer(X, n_h)
print("A1", A1)

Z_out = dense_layer(A1, n_y, activation = None)
print("Z_out", Z_out)


y_pred = tf.argmax(Z_out, axis=1, output_type=tf.int32)
print("y_pred", y_pred)

match = tf.equal(y_pred, y)
print("match", match)

accuracy = tf.reduce_mean(tf.cast(match, tf.float32))
print("accuracy", accuracy)

loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_oh, logits=Z_out)
print("loss", loss)

cost = tf.reduce_mean(loss)

op = tf.train.AdamOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(100):
        _, cost_ = sess.run([op, cost], feed_dict = {X: X_train, y: y_train})
        
        print(cost_)    
    
    train_accuracy = sess.run(accuracy, feed_dict = {X: X_train, y: y_train})
    print("Test accuracy: ", train_accuracy)
    
    test_accuracy = sess.run(accuracy, feed_dict = {X: X_test, y: y_test})
    print("Test accuracy: ", test_accuracy)

X Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
y Tensor("Placeholder_1:0", shape=(?,), dtype=int32)
y_oh Tensor("one_hot:0", shape=(?, 10), dtype=float32)
A1 Tensor("Relu:0", shape=(?, 100), dtype=float32)
Z_out Tensor("add_1:0", shape=(?, 10), dtype=float32)
y_pred Tensor("ArgMax:0", shape=(?,), dtype=int32)
match Tensor("Equal:0", shape=(?,), dtype=bool)
accuracy Tensor("Mean:0", shape=(), dtype=float32)
loss Tensor("softmax_cross_entropy_with_logits/Reshape_2:0", shape=(?,), dtype=float32)
2.3459146
2.2371297
2.137779
2.0465338
1.9619101
1.8822763
1.806164
1.732392
1.6602029
1.5892324
1.519485
1.4512701
1.3849909
1.3210576
1.2598597
1.2016159
1.1464094
1.0942796
1.0452254
0.99921834
0.95622176
0.91613775
0.8788108
0.84405243
0.81169146
0.7815399
0.753449
0.72729766
0.7029654
0.6803362
0.6592973
0.63972545
0.62150574
0.604525
0.58866763
0.5738581
0.56000936
0.5470519
0.5349072
0.5235027
0.51278293
0.50268984
0.49318582
0.4842301
0.47578558
0.4678097
0.46026015
0.4531031
0.4